## <center> **<span style="color: red">NLP</span> steps and explnation.**

Here we will use <span style="color: #ffd21e">**Hugging face**</span> trasformers pre trained models becuase it'll take so much effort to label<br>
all of our text columns "<ins>Comments, Describtion, Title</ins>" and also we will extract categorical and numirecal<br>
features from the text columns.<br><br>

##### **Steps in detail:**
<ul>
    <li>Extract categorical features from text columns sush as:<ol> 
        <li>Contains emojies
        <li>Positive or negative
        <li>Language
        <li>Video stats range</ol>
    <li> Doing analysis for most common words in comments and titles (Word cloud)
    <li> Extracing the length of the comments, descriptions & titles
    <li> One hot encoding
    <li> Stemming the data and tokenizing it
    <li> Saving those new data into new <strong>one</strong> Df to use in the final ML model

##### **Important note:**
The data in this analysis may be a little bit biased becuase all comments is made to be at least more than <br>
$50$.

## <center> **Importing the <span style="color: red">Packeges</span>**<br>

In [65]:
import re
import os
import ast
import time
import nltk
import emoji
import pickle
import sqlite3
import svgwrite
import numpy as np
import pandas as pd
import transformers
from PIL import Image
import huggingface_hub
from nltk.stem import wordnet
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from IPython.display import set_matplotlib_formats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from wordcloud import (WordCloud,
                       STOPWORDS,
                       ImageColorGenerator)

In [2]:
with open("../Data analysis/functions/z-score.pickle", "rb") as f:
    z_score = pickle.load(f)

huggingface_hub.notebook_login()
set_matplotlib_formats('pdf', 'svg')

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_8952\2025956571.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center> **Reading the <span style="color: red">data</span>**

In [3]:
TEXT_COLUMNS = ["title", "comments", "description", "channelTitle", "about"]

con = sqlite3.connect('../database.db')

df = pd.read_sql_query("""   
                        
                        SELECT *
                        FROM base_videos AS bv

                        INNER JOIN base_channels  AS bc ON
                            bc.channel_name = bv.channelTitle

                        INNER JOIN comments  AS c ON
                            c.video_id = bv.video_id""", con)

con.close()

df = df.T.drop_duplicates().T # dropping duplicated cols

In [4]:
df.sample(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,duration_in_minutes,subscribers,total_views,date,video_count,about,country,comments,like_counts,reply_counts
758,A7luS6IioBk,Roplex Gaming,Trading MEGA NAGA DRAGON (HUGE OVER PAY ADOPT ME),----------------------------------------------...,"['roblox', 'roplex gaming', 'adopt me', 'roblo...",2023-03-03 21:06:32+00:00,61007,1178,281,hd,9.75,650000,189972458,2019-05-11 00:00:00,929,I like to do adopt me trading videos on roblox...,US,slayyyyyyyy,0,0
4757,vvnjeMtrZvY,Yousef Gaming,12 قفل #6 | جزء جديد خرافي 😻🔥 !!,لا تشغلك المقاطع عن الصلوات 💖\nصلوا على محمد ﷺ...,"['يوسف احمد', 'يوسف', 'احمد', 'يوسف جيمنج', 'y...",2022-07-16 14:00:53+00:00,56921,4607,177,hd,12.101562,1790000,95599249,2016-11-05 00:00:00,270,سلام عليكم حبايبي في قناة الألعاب\nهنا في Yous...,SA,♡,1,0
1954,o3D9g0ohRIo,Gaming Insects,😂I Became A Giant Alex To Fooled This Girl in ...,😂I Became A Giant Alex To Fooled This Girl in ...,['Trolling My New Girlfriend as GIANT ALEX in ...,2023-03-02 07:30:08+00:00,144703,5977,628,hd,20.1875,694000,79491185,2019-08-22 00:00:00,415,Guys Im Trying My Best Ki App Logo Ko Bohot En...,IN,Yes,1,0
4553,uWpup9oq54w,Stealth17 Gaming,The German Fleet Arrives - Ultimate Admiral Dr...,The German fleet suddenly showed up in the Asi...,"['ultimate admiral dreadnoughts', 'ultimate ad...",2023-02-20 16:00:36+00:00,13260,691,55,hd,45.15625,102000,41969216,2014-08-18 00:00:00,3844,Calm & collected commentaries on strategy game...,NL,"Wow, that was one big slaughter !!! I can't aw...",4,0
1021,-iojQuqlWkU,DK Gaming,İLK ÇOCUKLAR ! (THE SIMS 4 100 BEBEK CHALLENGE...,✅ DK Gaming ten selamlar ben Duygu Köseoğlu bu...,"['dkgaming', 'duygu', 'duygu köseoğlu', 'ücret...",2022-12-05 12:00:11+00:00,30649,1335,189,hd,40.3125,252000,54565589,2019-03-07 00:00:00,515,Daha fazla oyun daha fazla eğlence! Duygu Köse...,TR,1 orada age up var neden yapmıyon 2 kurallar ç...,0,1


##  <center> **Extract <span style="color: red">categorical</span> features**

#### *Extract emojies count per text*

In [5]:
comments_emojis_counts: list = []
title_emojis_counts: list = []
desc_emojis_counts: list = []

for comment, title, desc in zip(df["comments"], df["title"], df["description"]):
    
    comments_emojis_count: int = 0
    title_emojis_count: int = 0
    desc_emojis_count: int = 0
    
    for comment_char, title_char, desc_char in zip(comment, title, desc):
        
        if emoji.is_emoji(comment_char):
            comments_emojis_count += 1
            
        if emoji.is_emoji(title_char):
            title_emojis_count += 1
            
        if emoji.is_emoji(desc_char):
            desc_emojis_count += 1
    
    comments_emojis_counts.append(comments_emojis_count)
    title_emojis_counts.append(title_emojis_count)
    desc_emojis_counts.append(desc_emojis_count)

In [6]:
df["comments_emojis_count"] = comments_emojis_counts
df["title_emojis_count"] = title_emojis_counts
df["desc_emojis_count"] = desc_emojis_counts

df.sample(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,...,date,video_count,about,country,comments,like_counts,reply_counts,comments_emojis_count,title_emojis_count,desc_emojis_count
4076,__g5jjC-Jf8,Vista Gamer,ROBANDO COCHES de POLICÍA en GTA 5,En el video de hoy robare coches de policía en...,"['gta 5', 'gta v', 'grand theft auto 5', 'gran...",2023-01-30 20:00:09+00:00,1509153,26013,553,hd,...,2020-02-10 00:00:00,260,CLICK en SUSCRIBIRSE!!\n\nSubo contenido relac...,AR,"muy buen contenido pero,hazlo sin vida infinit...",1,0,0,0,0
3325,p-7c0M0DtlA,Teches Gaming,CEI MAI BUNI JUCATORI DE VALORANT DIN ROMANIA ...,Pur și simplu suntem prea buni. \n\nTwitch Adr...,"['adriivonb', 'adri', 'adrii', 'adriana', 'twi...",2023-01-24 17:13:06+00:00,18514,1461,55,hd,...,2019-05-14 00:00:00,66,,RO,Mai mult val te rog,0,0,0,0,0
4620,MzkZyTV85-g,BUDI01 GAMING,"Masuk Global Ketemu Ciwi, Eh! Malahan Jumpa Ku...",Join Membership : \nhttps://www.youtube.com/c...,"['budi01gaming', 'budi01']",2023-01-21 12:27:04+00:00,1635097,63521,2691,hd,...,2019-07-19 00:00:00,297,Contact us :\n\nInstagram : budisembrenget\nht...,ID,Always Random Your Life Budi,0,0,0,0,0
4747,vvnjeMtrZvY,Yousef Gaming,12 قفل #6 | جزء جديد خرافي 😻🔥 !!,لا تشغلك المقاطع عن الصلوات 💖\nصلوا على محمد ﷺ...,"['يوسف احمد', 'يوسف', 'احمد', 'يوسف جيمنج', 'y...",2022-07-16 14:00:53+00:00,56921,4607,177,hd,...,2016-11-05 00:00:00,270,سلام عليكم حبايبي في قناة الألعاب\nهنا في Yous...,SA,ههههههههههههههههههههههههههههههه العبة مصصممه ع...,0,0,0,2,1
827,xh3Y1Sa7MUg,Gaming Harry,The Medium - Story Explained,"Hello everyone, and welcome to the next instal...","['the medium story explained', 'the medium gam...",2023-03-02 15:00:42+00:00,108533,4360,192,hd,...,2020-05-18 00:00:00,464,Hello everyone! Welcome to my channel! My name...,GB,"Let’s face it, most of us watching never playe...",0,0,0,0,0


#### *Language detection*

In [ ]:
language_detector = transformers.pipeline("text-classification",
                                          model="papluca/xlm-roberta-base-language-detection",
                                          use_auth_token= True)

In [ ]:
%%timeit
print(language_detector("هذا أطول نص قد تقرأه على الإطلاق إنه طويل للغاية", max_length=128))

Looks like we will take a lot to <ins>just classify the language</ins> of each text at least: $30\times100$ second whitch equalls <span style="color: red">**50 minutes.**</span><br>
So we can't do translation for now becuase it at least may take **5 hours** to run so we will just use this language detetion and<br>
optimize it by just sellecting one text column to detect which will be **Description column** and use the detected languages to<br>
pridect video and comments language.<br><br>
But we will detect the language using the **country of this video**

In [7]:
df["country"].unique()

array(['GB', 'IN', 'TH', 'US', 'TW', 'CA', 'PH', 'SG', 'PL', 'PK', 'DE',
       'AU', 'BD', 'ID', 'EG', 'AR', 'TR', 'JP', 'VN', 'NP', 'KR', 'RU',
       'BG', 'IT', 'SA', 'MT', 'RO', 'BR', 'NL', 'SE', 'DZ', 'FR', 'FI',
       'MX', 'AE', 'HU'], dtype=object)

In [8]:
country_languages = {
    'DE': 'German',
    'US': 'English', 'PL': 'Polish',
    'SA': 'Arabic', 'NP': 'Nepali',
    'CA': 'English', 'ES': 'Spanish',
    'TR': 'Turkish', 'IN': 'Hindi',
    'EG': 'Arabic', 'GB': 'English',
    'MX': 'Spanish', 'BR': 'Portuguese',
    'PK': 'Urdu', 'FR': 'French',
    'VN': 'Vietnamese', 'ID': 'Indonesian',
    'AU': 'English', 'HU': 'Hungarian',
    'NL': 'Dutch', 'BG': 'Bulgarian',
    'JP': 'Japanese', 'SG': 'English', 
    'TH': 'Thai', 'PH': 'Tagalog',
    'MT': 'Maltese', 'PE': 'Spanish',
    'SE': 'Swedish', 'IT': 'Italian',
    'KR': 'Korean', 'TW': 'Chinese',
    'FI': 'Finnish', 'DZ': 'Arabic',
    'BD': 'Bengali', 'AR': 'Spanish'}

df["language"] = df["country"].replace(country_languages)

This way may not be so accurate in detecting languages becuase there are some **indian videos** specifically with english and<br>
because some commetns come in deffrint languages but at least this way is more accurate than `roberta-language-detection`<br>
model because this model sometimes come with wierd 

#### *Sentiment analysis*

In [9]:
sentiment_classifier = transformers.pipeline(model= "finiteautomata/bertweet-base-sentiment-analysis")
print(sentiment_classifier("Hello here in my analysis, Have a nice day !!"))

Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

[{'label': 'POS', 'score': 0.9883897304534912}]


In [9]:
def my_sentiment_classifier(text: str) -> np.int8:
    """This code takes a text and return you if it's positive
        or negative as 1 for positive, 0 for natural -1 for negative
        and -10 for unclassified.
    
    @params: a string text
    @return: 1, 0 or -1 in np.int8 dtype"""
    
    sentiment_type_encoder: dict = {"POS": 1,
                                    "NEG": -1,
                                    "NEU": 0,
                                    "unclassified": -10}
    
    try:
        sentiment_type: str = sentiment_classifier(text)[0]["label"]
        
    except:
        sentiment_type: str = "unclassified"
        
    return  np.int8(sentiment_type_encoder[sentiment_type])

In [ ]:
df["sentiments"] = df["Comments"][:5].apply(
    lambda x: my_sentiment_classifier(x))

In [ ]:
df[["sentiments", "Comments"]]

#### *Video stats range*

We will classify each column of **[Comments, Likes, Views]** into 6 groups manually like:
<ul>
    <li>  $1\sim3,000$ Views
    <li>  $3,000\sim10,000$ Views
    <li>  $10,000\sim50,000$ Views
    <li>  $50,000\sim100,000$ Views
    <li>  $100,000\sim300,000$ Views
    <li>  $300,000\sim\infty$

We will see the max calues of each column so we can optimize it with the best dtype for it.

In [10]:
print(f" The max value of view count column is: {df['viewCount'].astype(int).max()}\n",
      f"The max value of like count column ias: {df['likeCount'].astype(int).max()}\n",
      f"The max value of comment count column is: {df['commentCount'].astype(int).max()}\n",
      f"The max value of subscribers column is: {df['subscribers'].astype(int).max()}\n")

 The max value of view count column is: 67652856
 The max value of like count column ias: 1164572
 The max value of comment count column is: 37636
 The max value of subscribers column is: 32000000



In [11]:
df = df.astype({"commentCount": np.uint16, "viewCount": np.uint32,
                "likeCount": np.uint16, "subscribers": np.uint32})

df.describe()

,viewCount,likeCount,commentCount,subscribers,comments_emojis_count,title_emojis_count,desc_emojis_count
count,6.078000e+03,6078.000000,6078.000000,6.078000e+03,6078.000000,6078.000000,6078.000000
mean,6.060396e+05,9996.761270,796.926621,1.552624e+06,0.668476,0.161731,0.125041
std,3.939468e+06,13534.257747,2641.105774,3.037447e+06,2.880683,0.495166,0.411487
min,2.186000e+03,1.000000,51.000000,6.720000e+03,0.000000,0.000000,0.000000
25%,3.338550e+04,1517.250000,128.250000,1.980000e+05,0.000000,0.000000,0.000000
50%,9.777500e+04,4016.000000,257.000000,4.985000e+05,0.000000,0.000000,0.000000
75%,3.112778e+05,11301.000000,628.000000,1.690000e+06,0.000000,0.000000,0.000000
max,6.765286e+07,63521.000000,37636.000000,3.200000e+07,64.000000,5.000000,4.000000


In [12]:
df["cat_view_count"] = pd.cut(df['viewCount'],
                         bins=[1, 3_000, 10_000, 50_000, 100_000, 300_000, 999_999_999_999],
                         labels=["from 1 to 3,000", "from 3,000 to 10,000",
                                 "from 10,000 to 50,000", "from 50,000 to 100,000",
                                 "from 100,000 to 300,000", "more than 300,000"])

df["cat_comment_count"] = pd.cut(df['commentCount'],
                         bins=[50, 100, 150, 200, 400, 600, 999_999_999_999],
                         labels=["from 50 to 100", "from 100 to 150",
                                 "from 150 to 200", "from 200 to 400",
                                 "from 400 to 600", "more than 600"])

df["cat_like_count"] = pd.cut(df['likeCount'],
                         bins=[1, 1_000, 5_000, 10_000, 50_000, 150_000, 999_999_999_999],
                         labels=["from 1 to 1,000", "from 1,000 to 5,000",
                                 "from 5,000 to 10,000", "from 10,000 to 50,000",
                                 "from 50,000 to 150,000", "more than 150,000"])

In [13]:
df[["cat_view_count", "cat_comment_count", "cat_like_count",
    "viewCount", "commentCount", "likeCount"]].sample(5)

,cat_view_count,cat_comment_count,cat_like_count,viewCount,commentCount,likeCount
3775,"more than 300,000",more than 600,"from 10,000 to 50,000",772444,906,38741
3132,"from 10,000 to 50,000",from 50 to 100,"from 1,000 to 5,000",22361,61,3720
5057,"from 100,000 to 300,000",from 50 to 100,"from 1,000 to 5,000",154774,89,3850
5015,"from 50,000 to 100,000",from 400 to 600,"from 1,000 to 5,000",86072,487,3136
2236,"from 100,000 to 300,000",more than 600,"from 10,000 to 50,000",263801,952,26395


#### *Text columns length*

In [49]:
for col in TEXT_COLUMNS:
    df[f"{col}_length"] = df[col].str.len()

## <center><strong>Text visualiztion with<span style = "color: red"> WordCloud</span></strong></center>

In [44]:
coloring = np.array(Image.open("../imgs/youtube_gaming_logo.png"))
stopwords = set(STOPWORDS)

comments = ''.join(df["comments"][df["language"] == "English"].values.flatten())

In [45]:
comments_wc = WordCloud(background_color= "white", max_words= 300, mask= coloring,
               stopwords= stopwords, max_font_size= 80, random_state= 42,
               font_path= '../Data analysis/assets/fonts/FranklinGothic.ttf',
               collocations=False)

image_colors = ImageColorGenerator(np.array(coloring))
comments_wc.generate(comments)
comments_wc = comments_wc.recolor(color_func= image_colors)

comments_svg = comments_wc.to_svg(embed_font=True)

# Save the SVG code to a file
with open("../plots/comments_word_cloud.svg", "w", encoding="utf-8") as f:
    f.write(comments_svg)
    
fig, ax = plt.subplots(1)

ax.imshow(comments_wc, interpolation="bilinear")
ax.set_axis_off()

plt.title("Comments word cloud")
fig.set_size_inches(10, 8)
plt.savefig("../plots/comments_word_cloud.png")
plt.show()

ValueError: We need at least 1 word to plot a word cloud, got 0.

Now we will make a word cloud for **video titles data**

In [40]:
popular_emojis = ["😊", "😍", "😘", "😜", "😝", "😂", "😭", "😡", "😠", "😩", "😫", "😞", "😟", "😢", "😥", "😰", "😱",
                       "😳", "😷", "👍", "👎", "👌", "👏", "🙌", "👋", "💪", "🙏", "❤️", "💔", "💕", "💖", "💘", "💙", "💚",
                       "💛", "💜", "💯", "🔥", "🌟", "✨", "⭐", "🌈", "🌺", "🍕", "🍔", "🍟", "🍦", "🍭", "🍩", "🍪", "🍺",
                       "🍻", "🍷", "🍸", "🎂", "🎁", "🎉", "🎊", "🎈", "🎵", "🎶", "🎼", "🎧", "🎤", "🎸", "🎹", "🎺", "🎷",
                       "🎻", "🎬", "🎥", "🎦", "📷", "📹", "📺", "📻", "💻", "📱", "💡", "🔑", "🔨", "🔥", "💰", "💳", "💼",
                       "📅", "📆", "📈", "📉", "📊", "📋", "📎", "📏", "📐", "🔒", "🔓", "🔍", "🔎", "🚀", "🚑", "🚒", "🚓",
                       "🚕", "🚗", "🚙", "🚚", "🚢", "🚤", "🚲", "🚶", "🚶‍♀️", "🏃", "🏃‍♀️", "⚽", "🏀", "🏈", "🎾", "🏐", "🏉",
                       "🎱", "🏓", "🏸", "🥊", "🥋", "🎮", "🕹️", "🎲", "🃏"]


for emoji in popular_emojis:
    stopwords.add(emoji)
    
titels = ''.join(df["title"][df["language"] == "English"].values.flatten())

In [41]:
coloring = np.array(Image.open("../imgs/joystick.png"))

titels_wc = WordCloud(background_color= "white", max_words= 2000, mask= coloring,
               stopwords= stopwords, max_font_size= 80, random_state= 42,
               font_path= '../Data analysis/assets/fonts/FranklinGothic.ttf')

titels_wc.generate(titels)
image_colors = ImageColorGenerator(np.array(coloring))
titels_wc = titels_wc.recolor(color_func= image_colors)


fig, ax = plt.subplots(1)

ax.imshow(titels_wc, interpolation= "bilinear")
ax.set_axis_off()

titels_svg = titels_wc.to_svg(embed_font=True)

with open("../plots/videos_titels_word_cloud.svg", "w", encoding="utf-8") as f:
    f.write(titels_svg)

plt.title("Video titles word cloud")
fig.set_size_inches(10, 8)
plt.savefig("../plots/videos_titels_word_cloud.png")
plt.show()

ValueError: We need at least 1 word to plot a word cloud, got 0.

## <center><strong><span style = "color: red">NLP</span> preprocessing

#### *Removing stop words*

In [29]:
from nltk.corpus import stopwords
# nltk.download()

nltk.download('punkt', force=True)
nltk.download('wordnet', force= True)
nltk.download('stopwords', force=True)
nltk.download('stopwords-hi', force=True)
nltk.download('stopwords-ar', force=True)
nltk.download('averaged_perceptron_tagger', force=True)


en_stopwords = set(stopwords.words('english')) 
ar_stopwords = set(stopwords.words('arabic')) 
# hi_stopwords = set(stopwords.words('hindi')) 

all_stopwords = en_stopwords.union(ar_stopwords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Error loading stopwords-hi: Package 'stopwords-hi' not
[nltk_data]     found in index
[nltk_data] Error loading stopwords-ar: Package 'stopwords-ar' not
[nltk_data]     found in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [20]:
def stopwords_dropper(words: list, stopwords: set) -> list:
    
    # Removing stop words from unalphabetical chars
    filtered_words = [re.sub(r"[\W_]", "", word) for word in words
                      if not word in stopwords]
    
    filtered_words = list(filter(lambda item: item is not "", filtered_words))
    return  filtered_words


for col in TEXT_COLUMNS:
    df[f"{col}_tokens"] = df[col].apply(lambda text: nltk.word_tokenize(text.lower()))
    df[f"{col}_tokens"] = df[f"{col}_tokens"].apply(lambda text: stopwords_dropper(text,
                                                                     all_stopwords))

df["comments_tokens"]

<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\FreeComp\AppData\Local\Temp\ipykernel_8952\3746631438.py:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  filtered_words = list(filter(lambda item: item is not "", filtered_words))


0       [msq, suffer, irreparable, emotional, damage, ...
1       [even, joke, description, elezen, forgets, dus...
2                                                  [cool]
3       [damn, d, love, able, enjoy, ff14, universe, r...
4       [emotional, trauma, way, floorior, truer, word...
                              ...                        
6073    [يوسف, الحلقه, الجايه, روح, الكهف, اشتغل, بتلق...
6074                                         [يوسف, بحبك]
6075    [يوسف, اذا, سويث, مهمة, تلسكوب, فلازم, تروح, ل...
6076    [يوسف, العب, dying, light, 2, اللعبة, كثير, حل...
6077    [يوسف, شكرا, هاذي, الفيديوهات, الجميله, اتمنى,...
Name: comments_tokens, Length: 6078, dtype: object

#### *Part-of-speech (POS) Tagging*

In [21]:
for col in TEXT_COLUMNS:
    df[f"{col}_pos_tags"] = df[f"{col}_tokens"].apply(lambda words: nltk.pos_tag(words))

df["about_pos_tags"]

0       [(pg, NN), (team, NN), (variety, NN), (gaming,...
1       [(pg, NN), (team, NN), (variety, NN), (gaming,...
2       [(pg, NN), (team, NN), (variety, NN), (gaming,...
3       [(pg, NN), (team, NN), (variety, NN), (gaming,...
4       [(pg, NN), (team, NN), (variety, NN), (gaming,...
                              ...                        
6073    [(سلام, JJ), (عليكم, NNP), (حبايبي, NNP), (قنا...
6074    [(سلام, JJ), (عليكم, NNP), (حبايبي, NNP), (قنا...
6075    [(سلام, JJ), (عليكم, NNP), (حبايبي, NNP), (قنا...
6076    [(سلام, JJ), (عليكم, NNP), (حبايبي, NNP), (قنا...
6077    [(سلام, JJ), (عليكم, NNP), (حبايبي, NNP), (قنا...
Name: about_pos_tags, Length: 6078, dtype: object

#### *Lemmatization and dropping duplicated words*

In [22]:
def get_wordnet_pos(treebank_tag: str) -> str:
    
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

In [30]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = []
lemmatized_words_group = []

for col in TEXT_COLUMNS:
    for index, row in df.iterrows():
        for token, pos_tag in zip(row[f"{col}_tokens"], row[f"{col}_pos_tags"]):

            wordnet_pos = get_wordnet_pos(pos_tag[1])
            lemmatized_words_group.append(lemmatizer.lemmatize(token, pos= wordnet_pos))
            lemmatized_words_group = list(set(lemmatized_words_group)) # Dropping duplicates


        lemmatized_words.append(lemmatized_words_group)
        lemmatized_words_group = [] # clearing this list
    
    df[f"{col}_tokens"] = lemmatized_words

AttributeError: module 'nltk.stem.wordnet' has no attribute 'ADJ'

In [ ]:
df["channelTitle_tokens"][:5]

#### *One hot encoding*

In [31]:
df["language"] = df["language"].astype("category").cat.codes
df["definition"] = df["definition"].astype("category").cat.codes
# df["sentiments"] = df["sentiments"].astype("category")

df["cat_view_count"] = df["cat_view_count"].replace({"from 1 to 3,000": 1, "from 3,000 to 10,000": 2,
                                                     "from 10,000 to 50,000": 3, "from 50,000 to 100,000": 4,
                                                     "from 100,000 to 300,000": 5, "more than 300,000": 6})

df["cat_like_count"] = df["cat_like_count"].replace({"from 1 to 1,000": 1, "from 1,000 to 5,000": 2,
                                                     "from 5,000 to 10,000": 3, "from 10,000 to 50,000": 4,
                                                     "from 50,000 to 150,000": 5, "more than 150,000": 6})

df["cat_comment_count"] = df["cat_comment_count"].replace({"from 50 to 100": 1, "from 100 to 150": 2,
                                                           "from 150 to 200": 3, "from 200 to 400": 4,
                                                           "from 400 to 600": 5, "more than 600": 6})

In [32]:
df[["language", "definition", "cat_view_count",
    "cat_like_count", "cat_comment_count"]].head()

,language,definition,cat_view_count,cat_like_count,cat_comment_count
0,6,0,3,2,4
1,6,0,3,2,4
2,6,0,3,2,4
3,6,0,3,2,4
4,6,0,3,2,4


## <center><strong><span style = "color: red">Text</span> classification

In [61]:
X = df[["language", "definition", "title_tokens", "about_tokens",
        "description_tokens", "comments_tokens", "comments_length",
        "description_length", "about_length", "title_length",
        "channelTitle_length"]]

y = df[["cat_like_count", "cat_view_count", "cat_comment_count"]]

X.sample(5)

,language,definition,title_tokens,about_tokens,description_tokens,comments_tokens,comments_length,description_length,about_length,title_length,channelTitle_length
351,6,0,"[runner, vs, deadly, bus, gta5]","[welcome, get, good, gaming, make, best, lan, ...","[escape, cops, gta5, https, youtubexi6nr3gv5ho...","[life, changed]",18,653,90,29,15
3612,6,0,"[8, crazy, things, old, lol, players, remember]","[league, legends, jeremy, gaming, curios, vide...","[league, legends, lol, 8, crazy, things, old, ...","[picked, game, late, s1, fun, look, back, mome...",77,1968,101,49,22
2789,6,0,"[predator, orion, 7000, upgradeable, liquid, c...","[want, see, ve, cooking, new, gaming, tech, un...","[treat, new, gaming, pc, supreme, taskmasterin...","[bought, 12th, gen, 5000, ve, seen, alarming, ...",686,528,153,57,15
299,27,0,"[buying, 1, million, rolls, royce, minecraft]","[pro, players, subscribe, hs, gaming, business...","[vlogging, channel, msvlogs35, little, brother...",[boat],4,394,107,45,9
1504,6,0,"[comparing, ranking, every, future, paradox, p...","[false, swipe, gaming, aims, make, quality, vi...","[compare, every, future, paradox, mon, origina...","[love, cause, nt, caught, generation, pokemon]",69,766,159,71,18


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 1/ 5, random_state= 99)

In [68]:
y_train

,cat_like_count,cat_view_count,cat_comment_count
191,2,5,4
2955,1,3,4
1127,2,4,4
2160,1,3,4
4227,2,2,2
...,...,...,...
1768,2,5,1
1737,2,3,2
3240,3,5,6
5305,1,2,2


#### *XGBoost classfier*

In [67]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier()),
])

# Fit the model and make predictions
text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

# Evaluate model performance
print(classification_report(y_test, y_pred))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31], got [1 2 nan 2 3 nan 3 nan 3 4 nan 4 nan 4 5 nan 5 nan 5 6 nan 6 nan 6 nan 6
 nan 6 nan 6 nan 6]